In [ ]:
import os
import pandas as pd
import numpy as np
import glob
from src import generate_annot_imgs

In [ ]:
def seconds_to_hours_minutes(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    return f"{hours} hours, {minutes} minutes"

# Example usage
total_seconds = (54*60+50+3*60+59)*199
total_seconds = (46*60+31+4*60)*129
total_seconds = (1*60*60+48*60+2*60+51)*74
result = seconds_to_hours_minutes(total_seconds)
print(result)

In [ ]:
# generating OBB boxes for MERR coordinates
# merr_filepath = r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years_id.csv"
# merr_database = pd.read_csv(merr_filepath, index_col=0)
MER_cord = pd.read_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years.csv", low_memory=False)
print(MER_cord.shape) # (231609, 17) - without 2019 
MER_cord = MER_cord.iloc[:,:-2]

In [ ]:
# assert we have the same fish id
n = MER_cord.groupby(by="name").cumcount()
nfpi = n.apply(lambda x: x+1)
tid = lambda x: str(x).split(".")[0]
MER_img_id = MER_cord.name.apply(tid)
MER_cord['fish_id'] = MER_img_id.astype(str) +"_"+ n.astype(str)
MER_cord['n_fish'] = nfpi

qaqc_paths = [r"Z:\Proj_Yolo\gobyfinder_yolov8\datasets\AUV_datasets\MER\2020_MERR_ratiohigherthan6.csv",
              r"Z:\Proj_Yolo\gobyfinder_yolov8\datasets\AUV_datasets\MER\2021_MERR_ratiohigherthan6.csv",
              r"Z:\Proj_Yolo\gobyfinder_yolov8\datasets\AUV_datasets\MER\2022_MERR_ratiohigherthan6.csv",
              r"Z:\Proj_Yolo\gobyfinder_yolov8\datasets\AUV_datasets\MER\2023_MERR_ratiohigherthan6.csv"]
qaqc_dfs = [pd.read_csv(path) for path in qaqc_paths]
qaqc_df = pd.concat(qaqc_dfs).rename(columns={"Unnamed: 0": "path"})
# # MER_cord.to_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years_id.csv")
fid = lambda x: os.path.basename(x).split(".")[0]
qaqc_df["fish_id"] = qaqc_df.path.apply(fid)
MER_cord = pd.merge(MER_cord, qaqc_df[["fish_type", "fish_id"]], on="fish_id", how="left")
MER_cord["image_id"] = MER_cord.name.apply(lambda x: x.split(".")[0])
print(MER_cord.shape)
MER_cord.head(2)

In [ ]:
MER_cord.fish_type.value_counts()

In [ ]:
## clean up MER
MER_cord.fish_type = MER_cord.fish_type.replace("goby goby", "goby")
MER_cord.fish_type = MER_cord.fish_type.replace("alewife alewife", "alewife")
MER_cord.fish_type = MER_cord.fish_type.replace("goby goby goby", "goby")
MER_cord.fish_type = MER_cord.fish_type.replace("notfish goby", "notfish")
MER_cord.fish_type = MER_cord.fish_type.replace("goby notfish", "notfish")
MER_cord.fish_type = MER_cord.fish_type.replace("goby alewife", "goby")
MER_cord.fish_type = MER_cord.fish_type.replace("goby alewife alewife", "alewife")
MER_cord.fish_type = MER_cord.fish_type.fillna("goby")
cls_dict = {"goby": 0, "alewife": 1, "notfish": 2, "other": 3}
MER_cord["cls"] = MER_cord.fish_type.apply(lambda x: cls_dict[x])
MER_filtered = MER_cord[(MER_cord.fish_type == "goby") | (MER_cord.fish_type == "alewife")]
MER_filtered = MER_filtered.copy()
''' 
cls
0    230919
3       423
1       219
4        48
Name: count, dtype: int64
'''
print(MER_cord.cls.value_counts())

# MER_cord.to_csv(r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years_id_fishtype_unfiltered.csv")
print(MER_cord.shape)
MER_cord.head(3)

In [ ]:
def from_merr_to_YOLO_OBB(merr_df, output_folder):
    timh = lambda x: int(x.split("x")[0].rstrip())
    timw = lambda x: int(x.split("x")[1].rstrip())
    for imid in merr_df.image_id.unique():
        lbl_fn = imid +".txt"
        df = merr_df[merr_df.image_id == imid]
        yr = str(df.year.unique()[0])
        cls = df.cls
        lbl_pth = os.path.join(output_folder,yr, lbl_fn)
        imh = df.img_size.apply(timh).unique()[0]
        imw = df.img_size.apply(timw).unique()[0]
        values = df[["BLX", "BLY", "TLX", "TLY", "TRX", "TRY", "BRX", "BRY"]]
        # convert to proportional
        values.iloc[:,[0,2,4,6]] = values.iloc[:,[0,2,4,6]]/imw
        values.iloc[:, [1,3,5,7]] = values.iloc[:, [1,3,5,7]]/imh
        # insert class id
        values.insert(0, "col0",cls)
        values.to_csv(lbl_pth, header=False, index=False, sep=" ")
output_folder = r"datasets\AUV_datasets\MER"
from_merr_to_YOLO_OBB(merr_df = MER_cord, output_folder = output_folder)

### dixplay MER boxes

In [ ]:
img_pths = glob.glob(r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\datasets\AUV_datasets\images\2021\*")
mlbl_pths = glob.glob(r"C:\Users\ageglio\ageglio-1\gobyfinder_yolov8\datasets\AUV_datasets\MER\2021\*")
alweife_imgs = MER_filtered[MER_filtered.fish_type == "alewife"].name.apply(lambda x: str(x).split(".")[0])
# idx = 50
lst = ['CI_1629400773_167', 'image_raw_04491933', 'image_raw_81116102', 'image_raw_81345213']

# name = str(alweife_imgs).split(".")[0]
# name = alweife_imgs[alweife_imgs.str.contains("CI_1629400773_167")].item()
name = "CI_1624914813_174"
simg, slbl = pd.Series(img_pths), pd.Series(mlbl_pths)
img, mlbl = simg[simg.str.contains(name)].item(), slbl[slbl.str.contains(name)].item()
generate_annot_imgs.disp_merr_bbox(img, mlbl)

## Checking label database with kailee's data

In [ ]:
# new_lbls_csv_pth = r"test_runs\detect\Run14+-alewife_annot_imgs_2023\Run14+-alewife_annot_imgs_2023_Labels.csv"
new_assmt_csv_pth = r"C:\Users\ageglio\Downloads\2023_assessment_confirmedfish_polynomialfit_corrected.csv"
lbl_database = r"Z:\__Organized_Directories_InProgress\Fish_annotations_based_on_YOLO\_MERR_coordinates_all_years_id_fishtype_unfiltered.csv"
df_a = pd.read_csv(new_assmt_csv_pth)
print(df_a.shape)
df_a = df_a[df_a.Fish_ID != 0]
print(df_a.shape)
df_id = pd.read_csv(lbl_database, index_col=0, low_memory=False)
print(df_id[df_id.year == 2023].shape)
df_id2 = df_id[df_id.image_id.isin(df_a.image_id)]
print(df_id2.shape)

In [ ]:
from shapely.geometry import Polygon
def calculate_iou_obb(bbox1, bbox2):
    """Calculates the IoU of two oriented bounding boxes.
    Args:
        bbox1 (list): A list of 8 coordinates representing the 4 corners of the first OBB.
        bbox2 (list): A list of 8 coordinates representing the 4 corners of the second OBB.
    Returns:
        float: The IoU value between the two OBBs.
    """
    # Create Shapely polygons from the bounding box coordinates
    poly1 = Polygon(bbox1)
    poly2 = Polygon(bbox2)
    # Calculate the intersection and union areas
    intersection_area = poly1.intersection(poly2).area
    union_area = poly1.area + poly2.area - intersection_area
    # Calculate and return the IoU
    return intersection_area / union_area
# Example usage
bbox1 = [(0, 0), (2, 0), (2, 1), (0, 1)]  # Axis-aligned for simplicity
bbox2 = [(1, 0.5), (3, 0.5), (3, 1.5), (1, 1.5)]
iou = calculate_iou_obb(bbox1, bbox2)
print(iou) 

In [ ]:
import cv2
import numpy as np
def get_rotation_from_obb(corners):
    """
    Calculates the rotation angle of an oriented bounding box from its corners.

    Args:
        corners (list): List of 4 corner points of the bounding box in the format [[x1, y1], [x2, y2], [x3, y3], [x4, y4]].

    Returns:
        float: Rotation angle in degrees converted to radians.
    """
    # Convert corners to numpy array
    corners = np.array(corners, dtype=np.float32)
    # Calculate minimum area rectangle
    rect = cv2.minAreaRect(corners)
    # Return the rotation angle in deg, then convert to radians
    print(rect[2])
    return rect[2] * (np.pi/180)
# Example usage
corners = [(1, 0.5), (3, 0.5), (3, 1.5), (1, 1.5)]
angle = get_rotation_from_obb(corners)
print("Rotation angle:", angle)

In [ ]:
year = "2023"
dfl = pd.read_csv(f"test_runs\\detect\\Run14+-alewife_annot_imgs_{year}\\Run14+-alewife_annot_imgs_{year}_Labels.csv", index_col=0)
dfp = pd.read_csv(f"test_runs\\detect\\Run14+-alewife_annot_imgs_{year}\\Run14+-alewife_annot_imgs_{year}_predictions.csv", index_col=0)

In [ ]:
imp = dfp.image_id
dflq = dfp[dfp.image_id.isin(imp)]
dflq.names.value_counts()